In [ ]:
import torch
import torch.nn as nn # This will make the weights and baises tensors to be apart of the NN
import torch.nn.functional as F # This gives us the activation functions
from torch.optim import SGD # S|GD (Stochastic Gradient Descent) this will be used to fit our NN to the data
import matplotlib.pyplot as plt # matplotlib and seaborn are used to plot graphs
import seaborn as sns

In [ ]:
from typing_extensions import Required
# In order to create a NN you start with a class
class BasicNN(nn.Module):
  def __init__(self):
    super().__init__() # This call the initalizatino funciton in the nn.Module
    # You are initalizing your weights and baises here and setting values to them
    self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad=False) # The requires_grad is set to false because we do not want to optimize this
    self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad=False)
    self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad=False)

    self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad=False)
    self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad=False)
    self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad=False)
    self.final_bais = nn.Parameter(torch.tensor(-16.), requires_grad=False)
  # This is for foward pass
  def forward(self, input):
    Z_value_Top_Relu = input * self.w00 + self.b00
    top_relu_func = F.relu(Z_value_Top_Relu)
    Scaled_Z_value_Top_Relu = top_relu_func * self.w01

    Z_value_bottom_Relu = input * self.w10 + self.b10
    bottom_relu_func = F.relu(Z_value_bottom_Relu)
    Scaled_Z_value_Bottom_Relu = bottom_relu_func * self.w11

    input_to_final_relu = Scaled_Z_value_Bottom_Relu + Scaled_Z_value_Top_Relu + self.final_bais
    output = F.relu(input_to_final_relu)
    return output


In [ ]:
input_doses = torch.linspace(start=0, end=1, steps=11)
input_doses

In [ ]:
model = BasicNN()
output_vals = model.forward(input_doses)

In [ ]:
sns.set(style="whitegrid")
sns.lineplot(x=input_doses, y=output_vals, color="green", linewidth=2.5)
plt.ylabel("Effectiveness")
plt.xlabel("Dose")

In [ ]:
class Basic_NN_train(nn.Module):
  def __init__(self):
    super().__init__()
    self.w00 = nn.Parameter(torch.tensor(1.7), requires_grad=False)
    self.b00 = nn.Parameter(torch.tensor(-0.85), requires_grad=False)
    self.w01 = nn.Parameter(torch.tensor(-40.8), requires_grad=False)

    self.w10 = nn.Parameter(torch.tensor(12.6), requires_grad=False)
    self.b10 = nn.Parameter(torch.tensor(0.0), requires_grad=False)
    self.w11 = nn.Parameter(torch.tensor(2.7), requires_grad=False)
    self.final_bias = nn.Parameter(torch.tensor(0.), requires_grad=True)

  def forward(self, input):
    z_top_relu = input * self.w00 + self.b00
    z_bottom_relu= input * self.w10 + self.b10
    relu_value_top = F.relu(z_top_relu)
    relu_value_bottom = F.relu(z_bottom_relu)
    Scaled_top_val = relu_value_top * self.w01
    Scaled_bottom_val = relu_value_bottom * self.w11
    final_z = Scaled_bottom_val + Scaled_top_val + self.final_bias
    return F.relu(final_z)


In [ ]:
model = Basic_NN_train()
output_vals = model.forward(input_doses)

In [ ]:
sns.set(style="whitegrid")
sns.lineplot(x=input_doses, y=output_vals.detach(), color="green", linewidth=2.5)
plt.ylabel("Effectiveness")
plt.xlabel("Dose")


In [ ]:
inputs = torch.tensor([0., 0.5, 1.0])
labels = torch.tensor([0.0, 1.0, 0.0])


In [ ]:
optimizer = SGD(model.parameters(), lr=0.1) # We use the Stochastic Gradient Descent to optimze or weights and biases using a learning rate of 0.1
print("Final Bias before training: " + str(model.final_bias.data))

In [ ]:
for epoch in range(100):
  total_loss = 0
  for iter in range(len(inputs)):
    pred = model.forward(inputs[iter])
    squared_residuals = (labels[iter] - pred)**2
    squared_residuals.backward() # This is used for backpropagation and the derivatives are added together (The reason for adding is in the link: https://www.google.com/search?q=why+does+.backward()+add+the+derivatives+in+pytorch&rlz=1C5CHFA_enUS979US980&oq=why+does+.backward()+add+the+derivatives+in+pytorch&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRifBTIHCAIQIRifBTIHCAMQIRifBTIHCAQQIRifBTIHCAUQIRifBTIHCAYQIRifBTIHCAcQIRifBTIHCAgQIRifBTIHCAkQIRifBdIBCTEzOTMxajFqN6gCALACAA&sourceid=chrome&ie=UTF-8)
    total_loss += float(squared_residuals)
  if total_loss < 0.0001:
    print("Num steps: " + str(epoch))
    break
  optimizer.step()
  optimizer.zero_grad() # This is used to clear the previous derivatives stored in .grad which contains the sum of the derivatives
  print("Step: " + str(epoch) + " Final bias: " + str(model.final_bias.data))

print("Final Bias before training: " + str(model.final_bias.data))

In [ ]:
output_vals = model.forward(input_doses)
sns.set(style="whitegrid")
sns.lineplot(x=input_doses, y=output_vals.detach(), color="green", linewidth=2.5)
plt.ylabel("Effectiveness")
plt.xlabel("Dose")
